# IMPORTS

In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.metrics import Precision, Recall, AUC
from sklearn.metrics import f1_score, precision_recall_curve
import json
from google.colab import runtime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DATASET

In [2]:
img_size = (224, 224)
channels = 3
batch_size = 1
img_shape = (img_size[0], img_size[1], channels)
categories = ["Normal","Osteopenia", "Osteoporosis"]
TfDatasetsPath = '/content/drive/MyDrive/CSEN-240-Project/TfDatasetsFull'

valDataset = tf.data.Dataset.load(TfDatasetsPath + "/validDataset")
testDataset = tf.data.Dataset.load(TfDatasetsPath + "/testDataset")

valDataset = valDataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
testDataset = testDataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Get Best Thresholds Function

In [3]:
def bestThresholds(model, valDataset):
    valid_labels = []
    for xBatch, yBatch in valDataset:
        valid_labels.extend(np.argmax(yBatch, axis=1))
    predictions = model.predict(valDataset)
    yTrue = np.eye(3)[valid_labels]
    yPred = predictions
    thresholds = {}

    for class_idx in range(3):
        precision, recall, thresholds_pr = precision_recall_curve(yTrue[:, class_idx], yPred[:, class_idx])

        f1_scores = 2 * (precision * recall) / (precision + recall)

        optimal_threshold_pr = thresholds_pr[np.argmax(f1_scores)]
        thresholds[class_idx] = optimal_threshold_pr

    return thresholds

# Get Best Thresholds for All Models

In [4]:
def convert_float32(obj):
    if isinstance(obj, dict):
        return {key: convert_float32(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_float32(item) for item in obj]
    elif isinstance(obj, np.float32):
        return float(obj)  # Convert numpy.float32 to native Python float
    else:
        return obj

In [5]:
kerasModelsPath = '/content/drive/MyDrive/CSEN-240-Project/KerasModels/'
modelNames = os.listdir(kerasModelsPath)
print("Models found: ", modelNames)

Models found:  ['Xception.keras', 'VGG16.keras', 'ResNet152V2.keras', 'DenseNet201.keras', 'InceptionResNetV2.keras', 'EfficientNetV2L.keras', 'NASNetLarge.keras', 'MobileNet.keras']


In [7]:
modelNames = ['MobileNet.keras']
for modelName in modelNames:
    name = modelName.split('.')[0]
    print("Processing: ", name)
    model = tf.keras.models.load_model(kerasModelsPath + modelName)
    bestThresholdsDict = bestThresholds(model, valDataset)
    bestThresholdsDict = convert_float32(bestThresholdsDict)
    with open('/content/drive/MyDrive/CSEN-240-Project/Thresholds/' + name + '.json', 'w') as jsonFile:
        json.dump(bestThresholdsDict, jsonFile)

Processing:  MobileNet
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step


In [8]:
runtime.unassign()